In [ ]:
# Library stuff
import pickle
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import requests
import pandas as pd

In [ ]:
# Psssssst.. secret stuff
CLIENT_SECRET_FILE = 'client_secret.json'
API_SERVICE_NAME = 'mybusiness'
API_VERSION = 'v4'
SCOPES = ['https://www.googleapis.com/auth/business.manage']


In [ ]:
# Input for the parameters
LANGUAGE = "nl-NL"
SUMMARY = input("\nWhat is the text summary of the post?\n\n")
ACTION = input("\nType: 'LEARN_MORE' or 'ORDER':\n\n")
URL = input("\nWhat is the landing page of the post?\n\n")
PHOTO = input("\nWhat is the url of the image you would like to include?\n\n")

In [ ]:
# Load, refresh of create token.pickle
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        cred = pickle.load(token)
if not cred or not cred.valid:
    if cred and cred.expired and cred.refresh_token:
        cred.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
        cred = flow.run_local_server()
    with open('token.pickle', 'wb') as token:
        pickle.dump(cred, token)
try:
    service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
    print('Service created succesfully')
    print(service)
except Exception as e:
    print(e)

BEARER = cred.token

In [ ]:
# Gather location info
response = requests.get("https://mybusinessaccountmanagement.googleapis.com/v4/accounts", headers={"Authorization": f"Bearer {BEARER}"})
print(response.status_code)
accounts = response.json()
print(accounts)

In [ ]:
# Gather all locations 
response = requests.get("https://mybusinessaccountmanagement.googleapis.com/v4/accounts/{YOUR_LOCATION_ID_YOU_JUST_COPIED}/locations", headers={"Authorization": f"Bearer {BEARER}"})
print(response.status_code)
locations = response.json()
# print(locations) 

In [ ]:
# Find all location IDs
shops = [i["name"] for i in locations["locations"]]
# print(shops)

In [ ]:
# Get the parameters in place
params = {
  "languageCode": LANGUAGE,
  "title": TITLE,
  "summary": SUMMARY,
  "callToAction": {
    "actionType": ACTION,
    "url": URL + "?utm_source=gmb&utm_medium=organic"
  },
  "media": [
    {
      "mediaFormat": "PHOTO",
      "sourceUrl": PHOTO,
    }
  ],
}

print(params)

{'languageCode': 'nl-NL', 'summary': '142', 'callToAction': {'actionType': 'update', 'url': 'gwe?utm_source=gmb&utm_medium=organic'}, 'media': [{'mediaFormat': 'PHOTO', 'sourceUrl': '2'}]}


In [ ]:
for shop in shops:
    url = f"https://mybusiness.googleapis.com/v4/{shop}/localPosts"
    make_post = requests.post(url, json=params, headers={"Authorization": f"Bearer {BEARER}"})
    # print(make_post.status_code)